In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random

In [ ]:
import torch
import numpy
import pandas
import gensim

In [ ]:
from gensim.models import KeyedVectors 
 


#word encodings

word2vec

In [ ]:
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP-Fnd/Unzipped/GoogleNews-vectors-negative300.bin',binary=True)

In [ ]:
dim=300

#EMBED

In [ ]:
def vectorize(sentence) :
  tokens = sentence.split(' ')
  x = torch.randn( 1,len(tokens),dim )
  for i in range( min(len(tokens),50) ) :
    try :
      vector = model[ tokens[i].lower() ]
      for j in range(dim) :
        x[0][i][j] = torch.tensor(vector[j])
    except Exception as e:
        pass 
  return x 

In [ ]:
def Give(frame,index) :
  titlev = vectorize(frame['title'][index])
  textv  = vectorize(frame['text'][index] )
  y = torch.zeros(1,1)
  if frame['Label'][index] == 'true' :
    y[0][0] = 1.0
  else :
    y[0][0] = 0.0
  return [ [titlev,textv]  , y ]

# Data loading

In [ ]:
#__________________________________________________FETCHING THE DATAFRAME__________________________________________________________________________


train_data_Fake=pandas.read_csv("/content/drive/MyDrive/NLP-Fnd/Fake.csv") #FAKE File Read
train_data_Tre=pandas.read_csv("/content/drive/MyDrive/NLP-Fnd/True.csv") #TRUE File Read
#Columns---->title	text	subject	date



train_data_Fake["Label"]="fake" #Fake Label
train_data_Tre["Label"]="true"  #True Label 



#Title  Body  Combine
# ___________________________COncatInating two DatafRames of True and Fake______________________

frame = train_data_Fake.append([train_data_Tre])
#Creating the text and title in one column concat
# frame["combined"] = frame["title"] +" "+frame["text"]


#_________________________________RANDOM SHUFFLING____________________________________________
frame = (frame.sample(frac = .1)).reset_index(drop=True)[["title","text","Label"]]


In [ ]:
frame[:100]

,title,text,Label
0,JUSTICE SCALIA Found Dead With ‘Pillow Over Hi...,It was just announced that there will be no au...,fake
1,Nearly 400 die as Myanmar army steps up crackd...,"COX S BAZAR, Bangladesh (Reuters) - Nearly 400...",true
2,Maxine Waters Hijacks Funeral To Bash Trump…Tu...,Maxine Waters got downright nasty at the funer...,fake
3,Senate passes measure that permanently repeals...,WASHINGTON (Reuters) - Senate Republicans pass...,true
4,SHARIA COMPLIANT SWIMSUITS: As Spring Arrives ...,"Make no mistake, the Muslim European invaders ...",fake
...,...,...,...
4485,LOL! RADIO STATION HOSTS Car Wash For Veterans...,An online radio station based out of Oklahoma ...,fake
4486,Another One Bites The Dust: GOP Candidate Han...,Another Republican has dropped out of the race...,fake
4487,Meet Leftist Vogue Fashion Editor Slammed on S...,A fashion editor for Vogue (see video below) h...,fake
4488,China's Xi tells India's Modi to safeguard pea...,BEIJING (Reuters) - China s President Xi Jinpi...,true


#Network


In [ ]:
def contanicate(f1,f2) :
  size_data = f1.shape[1]
  feature_size1 = f1.shape[2]
  feature_size2 = f2.shape[2]
  fcn = torch.zeros( (size_data) , 2*(feature_size1+feature_size2) ).to(device)

  for index in range(size_data) :
    indexv=0
    for dir in range(2) :
      for feature_row in range(feature_size1) :
        fcn[index][indexv] = f1[dir][index][feature_row]
        indexv=indexv+1
    for dir in range(2) :
      for feature_row in range(feature_size2) :
        fcn[index][indexv] = f2[dir][index][feature_row]
        indexv=indexv+1

  return fcn

In [ ]:
a = torch.zeros( 2,3 ,requires_grad = True)

In [ ]:
a

tensor([[0., 0., 0.],
        [0., 0., 0.]], requires_grad=True)

In [ ]:
b = Encoder_LSTM(1,1)

In [ ]:
list(b.parameters())

[Parameter containing:
 tensor([[-0.0567]], requires_grad=True), Parameter containing:
 tensor([[-0.7356]], requires_grad=True), Parameter containing:
 tensor([-0.1275], requires_grad=True), Parameter containing:
 tensor([0.5790], requires_grad=True)]

In [ ]:
k = Encoder_LSTM(1,1)

In [ ]:
list(k.parameters())

[Parameter containing:
 tensor([[0.0305]], requires_grad=True), Parameter containing:
 tensor([[0.0320]], requires_grad=True), Parameter containing:
 tensor([-0.9854], requires_grad=True), Parameter containing:
 tensor([-0.5705], requires_grad=True)]

In [ ]:
len( list(b1) )

8

In [ ]:

class Encoder_LSTM(torch.nn.Module) :
    def __init__(self,input_dim , output_dim , ISL = False ) :
        super(Encoder_LSTM,self).__init__()
        self.Input_dim = input_dim
        self.Output_dim = output_dim
        self.Is_Shallow = True
        self.Neuron = output_dim
        self.IsBiDir = 2
        self.ISL = ISL
        self.device =None
        self.atten_matrix = torch.zeros( self.Output_dim , self.Output_dim , requires_grad=True )
        self.Initialize()

    def Initialize(self) :
        self.LSTM = torch.nn.LSTM(input_size = self.Input_dim , hidden_size = self.Output_dim , num_layers = 1 ,bidirectional=True, batch_first = True  ,  dropout=0)
        #self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.device = device

    def Initialize_Hidden(self,batch_size) :
      return torch.zeros( 2 , batch_size , self.Neuron)

    def Initialize_Cell(self,batch_size) :
      return torch.zeros( 2 , batch_size , self.Neuron )

    def Encode(self,seq) :
        batch_size = seq.shape[0]
        #embeds =  embeddings( seq )
        ihs = self.Initialize_Hidden(batch_size)
        ics   = self.Initialize_Cell(batch_size)
        Latent_Feature = self.forward(seq,ihs,ics)
        return Latent_Feature

    def forward(self,seq,initial_hidden_state,intial_cell_state) :
        output , ( h_n , c_n) = self.LSTM(seq, ( initial_hidden_state , intial_cell_state ) )
        return h_n




In [ ]:
class DS_Learner_1HL(torch.nn.Module) :
    def __init__(self  ,input_dim , output_dim , hidden ,dropout=0 ) :
        super(DS_Learner_1HL,self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.hidden = hidden
        self.Initialize()

    def Initialize(self) :
        self.fc1 = torch.nn.Linear(self.input_dim,self.hidden) 
        self.sigmoid1 = torch.nn.Sigmoid() 
        self.fc2 = torch.nn.Linear(self.hidden,self.output_dim)
        self.sigmoid2 = torch.nn.Sigmoid()

    
    def forward(self,input_x) :
        lc1 = self.fc1(input_x)
        nl1 = self.sigmoid1(lc1)
        lc2 = self.fc2(nl1)
        nl2 = self.sigmoid2(lc2)
        return nl2



In [ ]:
class Wrapper(torch.nn.Module) :
  def __init__(self,ec1,ec2,cn) :
    super(Wrapper,self).__init__()
    self.enc1 = ec1
    self.enc2 = ec2
    self.cn  = cn 
  def forward(self,x) :
    hs1 = self.enc1.Encode(x[0]) 
    hs2 = self.enc2.Encode(x[1])
    hsc = contanicate(hs1,hs2)
    final = self.cn(hsc)
    return final

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


#setting art

In [ ]:
device = 'cpu'

In [ ]:
ems = 300
hs1=10
hs2 =15 
hl1s=10

In [ ]:
enc1 = Encoder_LSTM(ems,hs1)
enc2 = Encoder_LSTM(ems,hs2)
cn = DS_Learner_1HL( 2*(hs1+hs2) ,1, hl1s )

In [ ]:
w = Wrapper(enc1,enc2,cn).to(device)

In [ ]:
optimizer = torch.optim.Adam(w.parameters()   , lr=0.2)

In [ ]:
def Train1(NG,optimizer,frame,max_epoch=1) :
  length = frame.shape[0]
  size_batch = 49
  number_batch = int(length /size_batch)
  print("length ",length,"  size Batch ",size_batch)
  Loss_function = torch.nn.BCELoss() 


  for epoch in range(max_epoch) :
    print("----------------------------------EPOCH--------------------------------------",epoch)
    for bindex in range( 1 ) :
      loss = 0 
      for  i in range( bindex*size_batch , (bindex+1)*size_batch )  :
        [x,y] = Give(frame, i ) 
        pred = NG(x)
        loss = loss + Loss_function(pred,y)
        print("#",end='')
      loss = loss/size_batch
      optimizer.zero_grad()
      loss.backward() 
      print("\nepoch:",epoch ,"  iter:",bindex ,"  loss:",loss)
      optimizer.step()

      

In [ ]:
Train1(w,optimizer,frame,10)

length  4490   size Batch  49
----------------------------------EPOCH-------------------------------------- 0
#################################################
epoch: 0   iter: 0   loss: tensor(0.7004, grad_fn=<DivBackward0>)
----------------------------------EPOCH-------------------------------------- 1
#################################################
epoch: 1   iter: 0   loss: tensor(0.7753, grad_fn=<DivBackward0>)
----------------------------------EPOCH-------------------------------------- 2
#################################################
epoch: 2   iter: 0   loss: tensor(0.5918, grad_fn=<DivBackward0>)
----------------------------------EPOCH-------------------------------------- 3
#################################################
epoch: 3   iter: 0   loss: tensor(0.4686, grad_fn=<DivBackward0>)
----------------------------------EPOCH-------------------------------------- 4
#################################################
epoch: 4   iter: 0   loss: tensor(0.3234, grad_fn=<DivBa

In [ ]:
w( Give( frame,0 )[0] ) , Give(frame,0)[1] 

(tensor([[0.0200]], grad_fn=<SigmoidBackward>), tensor([[0.]]))

In [ ]:
for i in range(50) :
  print("original  :",frame['Label'][i],"            prediction:",w( Give(frame,i)[0] ),"   lf:",Give(frame,i)[1])